In [ ]:
import pandas as pd

In [ ]:
raw_severe_df = pd.read_csv('severe_weather_dataset.csv')

In [ ]:
# from https://stackoverflow.com/questions/41336756/find-the-closest-latitude-and-longitude
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav)) # 12742 = earth diameter km

# data is a list of dicts w/ 'lat', 'lon', and 'lct' (location number)
# v is a single dict w/ 'lat', 'lon'
def closest(data, v):
    mindist = 100000
    beststore = data[0]
    for datum in data:
        dist = distance(v['lat'],v['lon'],datum['lat'],datum['lon'])
        if dist < mindist:
            mindist = dist
            beststore = datum
    return beststore, mindist

In [ ]:
# Read in a list of store locations along with longitude and latitude.
# stores identified by location number, column lct_nbr
# latitude and longitude given by columns ltd_msr, lng_msr
store_df = pd.read_csv('path_to_your_stores_csv.csv')

In [ ]:
lctdict = []
for lct in set(store_df['lct_nbr']):
    lct_row = store_df[store_df['lct_nbr'] == lct].iloc[0]
    lctdict += [{'lat': lct_row['ltd_msr'], 'lon': lct_row['lng_msr'], 'lct': lct}]

In [ ]:
from geopy.geocoders import Nominatim
import tqdm
import numpy as np
geolocator = Nominatim(user_agent='myapplication')

# associate each weather event with a particular store location
failed = []
raw_severe_df['lct_nbr'] = [-1] * raw_severe_df.shape[0]
raw_severe_df['dist'] = [100000] * raw_severe_df.shape[0]
for iidx in tqdm.tqdm(range(raw_severe_df.shape[0])):
    row = raw_severe_df.iloc[iidx]
    if np.isnan(row['begin_lat']) or np.isnan(row['begin_lon']):
        if row['begin_location'] is None:
            try:
                # get last word of county name, eg. Litchfield from "Southern Litchfield"
                location = geolocator.geocode(row['cz_name'].split(' ')[-1] + ' ' + row['state'])
            except:
                failed += [iidx]
                continue
        else:
            try:
                location = geolocator.geocode(row['begin_location'] + ' ' + row['state'])            
            except:
                continue
            
        try:
            v = {'lat': location.latitude, 'lon': location.longitude}
        except:
            failed += [iidx]
            continue
    else:    
        v = {'lat': row['begin_lat'], 'lon': row['begin_lon']}
    closest_store, dist = closest(lctdict, v)
    raw_severe_df['lct_nbr'].iat[iidx] = closest_store['lct']
    raw_severe_df['dist'].iat[iidx] = dist

In [ ]:
# filter severe weather events so there is only one per day per location
from collections import defaultdict
maxit = -1
ccounts = defaultdict(int)
todrop = set()
for iidx in tqdm.tqdm(range(raw_severe_df.shape[0])):
    row = raw_severe_df.iloc[iidx]
    lct_date_df = raw_severe_df[(raw_severe_df['lct_nbr'] == row['lct_nbr']) & (raw_severe_df['begin_yearmonth'] == row['begin_yearmonth']) & (raw_severe_df['begin_day'] == row['begin_day'])]
    mindist = lct_date_df['dist'].min()
    final_ld_df = lct_date_df[lct_date_df['dist'] == mindist]
    mintime = final_ld_df['begin_time'].min()
    final_ld_df = final_ld_df[final_ld_df['begin_time'] == mintime]
    if final_ld_df.shape[0] > maxit:
        maxit = final_ld_df.shape[0]
        itit = final_ld_df
    if final_ld_df.shape[0] > 1:
        ccounts[(row['lct_nbr'], row['begin_yearmonth'], row['begin_day'])] += 1        

    # if we have filtered by mindist and min time and there are still multiple events, choose one at random
    final_ld_df = final_ld_df.sample(n = 1)    

    diffset = set(lct_date_df.index).difference(set(final_ld_df.index))
    todrop = todrop.union(diffset)

filtered_raw_df = raw_severe_df.drop(todrop)

In [ ]:
# Fill in date column to match with interactions dataset feed_date
filtered_raw_df['date'] = [""] * filtered_raw_df.shape[0]
for iidx in tqdm.tqdm(range(filtered_raw_df.shape[0])):
    row = filtered_raw_df.iloc[iidx]
    dateresult = str(row['begin_yearmonth'])[:4] + "-" + str(row['begin_yearmonth'])[4:6] + '-' + f"{row['begin_day']:02}"
    filtered_raw_df['date'].iat[iidx] = dateresult

In [ ]:
# for each location, fill in all calendar days with the number of days since a severe weather event
import datetime
filtered_raw_df['days_since'] = [0] * filtered_raw_df.shape[0]
new_rows = pd.DataFrame(columns=filtered_raw_df.columns)
mindate = min(filtered_raw_df['date'])
maxdate = max(filtered_raw_df['date'])
for lct_nbr in tqdm.tqdm(set(filtered_raw_df['lct_nbr'])):
    last_row_found = False
    last_row = None
    for ddate in pd.date_range(mindate,maxdate,freq='d'):
        datestr = str(ddate)[:10]
        row = filtered_raw_df[(filtered_raw_df['lct_nbr'] == lct_nbr) & (filtered_raw_df['date'] == datestr)]
        assert(row.shape[0] == 0 or row.shape[0] == 1)        

        # Date is present in df
        if row.shape[0] == 1:
            last_row_found = True
            last_row = row
            continue        

        # date is not present in df
        # fill in missing date with last df
        if last_row_found:
            new_row = last_row.copy()
            new_row['date'] = datestr
            new_row['days_since'] = datetime.datetime.strptime(datestr, "%Y-%m-%d").date() - datetime.datetime.strptime(last_row['date'].item(), "%Y-%m-%d").date()
            #new_rows = new_rows.append(new_row)
            new_rows = pd.concat([new_rows, new_row], ignore_index=True)
 
#new_rows = new_rows.append(filtered_raw_df, ignore_index=True)
new_rows = pd.concat([new_rows, filtered_raw_df], ignore_index=True)
new_rows.sort_values(by=['lct_nbr','date'], ascending=True, inplace=True)
new_days_since = new_rows['days_since']
for iidx in tqdm.tqdm(range(new_rows.shape[0])):
    if not type(new_rows['days_since'].iloc[iidx]) is int:
        new_days_since.iat[iidx] = new_rows['days_since'].iloc[iidx].days
new_rows['days_since'] = new_days_since

In [ ]:
filtered_raw_df = new_rows

In [ ]:
# Read in a list of interaction data.
# Customers have specified preferred/local store, which we use to match
# interaction df must include columns: 'store_nbr' (preferred/local store) and 'feed_date' (interaction date).
# store_nbr should match format of filtered_raw_df 'lct_nbr' column
# feed_date should match format of filtered_raw_df 'date' column
store_df = pd.read_csv('path_to_your_interactions_csv.csv')

In [ ]:
# merge store/weather dataset with interaction data
bigdf = interaction_df.merge(filtered_raw_df, how='left', left_on=['store_nbr', 'feed_date'], right_on=['lct_nbr', 'date'], suffixes=('_interactions', '_weather'))